<a href="https://colab.research.google.com/github/JulieChandlerDS/Data-Science-Portfolio/blob/main/Text_Generation_(RNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [ ]:
#Space Rats of the CCC by Harry Harrison
#path_to_file = tf.keras.utils.get_file('ccc.txt', 'https://raw.githubusercontent.com/JulieChandlerDS/spaceratsccc/main/ccc.txt')
#once and future king
#path_to_file = tf.keras.utils.get_file('theonceandfutureking.txt', 'https://raw.githubusercontent.com/JulieChandler-DS/onceandfutureking/main/onceandfutureking.txt')
#All your Language are Destroyed by us
#path_to_file = tf.keras.utils.get_file('RNN Fanfic Trainer.txt', 'https://raw.githubusercontent.com/JulieChandlerDS/ayladbu/main/RNN%20fanfic%20trainer.txt')
path_to_file = tf.keras.utils.get_file('Clownpost.txt', 'https://raw.githubusercontent.com/JulieChandlerDS/ayladbu/main/Clownpost.txt')

In [ ]:
#Read and decode txt for file compat
#encoding utf-8 seems to work for all txt files
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of   text: {len(text)} characters')

In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

In [ ]:
#View unique characters
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

In [ ]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [ ]:
ids = ids_from_chars(chars)
ids

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
chars = chars_from_ids(ids)
chars

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

In [ ]:
seq_length = 100

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 17926

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

In [ ]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

In [ ]:
model.summary()

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [ ]:
sampled_indices

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

In [ ]:
tf.exp(example_batch_mean_loss).numpy()

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS = 120

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/120
4/4 [==============================] - 24s 5s/step - loss: 4.2395
Epoch 2/120
4/4 [==============================] - 21s 5s/step - loss: 4.9218
Epoch 3/120
4/4 [==============================] - 21s 5s/step - loss: 3.9170
Epoch 4/120
4/4 [==============================] - 22s 5s/step - loss: 3.7160
Epoch 5/120
4/4 [==============================] - 21s 5s/step - loss: 3.5454
Epoch 6/120
4/4 [==============================] - 21s 5s/step - loss: 3.3099
Epoch 7/120
4/4 [==============================] - 21s 5s/step - loss: 3.1415
Epoch 8/120
4/4 [==============================] - 22s 6s/step - loss: 3.1070
Epoch 9/120
4/4 [==============================] - 21s 5s/step - loss: 3.0294
Epoch 10/120
4/4 [==============================] - 21s 5s/step - loss: 2.9736
Epoch 11/120
4/4 [==============================] - 21s 5s/step - loss: 2.9119
Epoch 12/120
4/4 [==============================] - 22s 5s/step - loss: 2.8376
Epoch 13/120
4/4 [==============================] - 22s 5s/st

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['"U should be my clown gf'])
result = [next_char]

for n in range(5000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)